要开发一个Web留言簿，下面将用python来开发这个程序的后台功能

1. 保存评论数据：使用标准模块shelve

In [ ]:
import shelve

DATA_FILE = 'guestbook.dat'

def save_data(name, comment, create_at):
    # 读取shelve文件
    database = shelve.open(DATA_FILE)
    # 如果数据库中没有comments列表，就新建一个
    if 'comments' not in database:
        comments = []
    else:
        comments = database['comments']
    # 将提交的数据添加到comments列表的开头
    comments.insert(0, {'name': name, 'comment': comment, 'create_at': create_at})
    # 更新数据库，但是为什么？
    database['comments'] = comments
    database.close()

2. 取出保存过的评论数据

In [ ]:
def load_data():
    database = shelve.open(DATA_FILE)
    comments = database.get('comments', [])
    database.close()
    return comments

3. 使用模板引擎显示页面：使用flask

In [ ]:
from flask import Flask, request, redirect, Markup
app = Flask(__name__)

@app.route('/')
def index():
    comments = load_data()
    comments_html = ''
    for comment in comments:
        comment_html = '''
        <div class="comment">
            <span class="create_at">[%s]</span>
            <span class="name">%s</span>:
            <span class="comment">%s</span>
        </div>
        ''' % (comment['create_at'], comment['name'], comment['comment'])
        comments_html += Markup(comment_html)
    return '''
    <html>
        <head>
            <title>Web留言簿</title>
            <style type="text/css">
                .comment {margin-bottom: 10px;}
                .create_at {color: #999;}
                .name {font-weight: bold;}
            </style>
        </head>
        <body>
            <h1>Web留言簿</h1>
            <div id="comments">
                %s
            </div>
            <h3>添加新评论</h3>
            <form action="/post" method="post">
                <div>姓名：<input type="text" name="name"></div>
                <div>评论：<textarea name="comment" rows="3" cols="40"></textarea></div>
                <div><input type="submit" value="提交"></div>
            </form>
        </body>
    </html>
    ''' % comments_html

if __name__ == '__main__':
    app.run()

4. 准备评论的目标URL地址：使用datatime模块

In [ ]:
from datetime import datetime

@app.route('/post', methods=['POST'])
def post():
    name = request.form.get('name')
    comment = request.form.get('comment')
    create_at = datetime.now()
    save_data(name, comment, create_at)
    return redirect('/')

5. 调整模板输出：两个模板过滤器

In [ ]:
@app.template_filter('datetime_fmt')
def datetime_fmt_filter(dt):
    return dt.strftime('%Y-%m-%d %H:%M:%S')

@app.template_filter('nl2br')
def nl2br_filter(s):
    return Markup(s.replace('\n', '<br>'))